<a href="https://colab.research.google.com/github/HssahSaad/Data_Science_AI/blob/main/RAG_Exam_Hssah_Alsherihi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [70]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dataset.zip
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Dataset.csv             



* Explore the dataset

In [71]:
import pandas as pd

In [72]:
# read the dataset
df = pd.read_csv('Dataset.csv')

In [73]:
df

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال
...,...,...
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [75]:
df.duplicated().sum()

0

In [76]:
df.isnull().sum()

,0
المخالفة,0
الغرامة,0


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [77]:
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [78]:
import os

In [79]:
directory = 'Markdown_file'
os.makedirs(directory,exist_ok = True)

In [80]:
for i in range(0,100):
  Violation = df["المخالفة"].iloc[i]
  Fine = df["الغرامة"].iloc[i]

  mark_content = f'**{Violation}**\n'
  mark_content = f'{Fine}\n'

  with open(f'{directory}/ traffic violations{i}.md', 'w', encoding='utf-8') as file:
        file.write(mark_content)

In [81]:
import markdown

markdown_viol = []

for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_viol.append(html_content)


In [82]:
markdown_viol[0]

'<p>الغرامة المالية 150 - 300 ريال</p>'

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [83]:
!pip install -qU langchain-text-splitters

In [84]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [85]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=130,
    chunk_overlap=20,
)

In [86]:
texts = text_splitter.create_documents(markdown_viol)

In [87]:
texts

[Document(metadata={}, page_content='<p>الغرامة المالية 150 - 300 ريال</p>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 150 - 300 ريال</p>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 1000 - 2000 ريال</p>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 150 - 300 ريال</p>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 300 - 500 ريال</p>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 500 - 900 ريال</p>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 500 - 900 ريال</p>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 150 - 300 ريال</p>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 100 - 150 ريال</p>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 3000 - 6000 ريال</p>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 100 - 150 ريال</p>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 1000 - 2000 ريال</p>'),
 Document(metadata={}, page_content='<p>الغرامة المالية 10

In [88]:
print(texts[0])
print(texts[1])

page_content='<p>الغرامة المالية 150 - 300 ريال</p>'
page_content='<p>الغرامة المالية 150 - 300 ريال</p>'


## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [89]:
!pip install langchain_community

In [90]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [91]:
! pip install langchain_chroma

In [92]:
from langchain_chroma import Chroma

In [93]:
!pip install sentence_transformers

In [94]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [95]:
embedding_function = SentenceTransformerEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")
db = Chroma.from_documents(texts, embedding_function, persist_directory="./chroma_db")

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [96]:
from langchain_core.prompts import PromptTemplate

In [97]:
prompt_template =(
    """
# التعليمات
* فقط أجب عن السؤال بإعطاء الإجابة الغرامة
* لا تجيب عن أي شيء خارج السياق
------

أجب عن السؤال بناءً على السياق التالي فقط:
تشابه: {similarity_Context}
السؤال : {Violation}
الجواب:

"""
)
prompt_template = PromptTemplate(
    template = prompt_template,
    input_variables=["similarity_Context","Violation"]
)


## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [98]:
!pip install langchain_groq

In [99]:
os.environ["GROQ_API_KEY"] = "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"

In [100]:
from langchain_groq import ChatGroq

#temperature --> to keep the answers in the context
llm = ChatGroq(temperature=0,  model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [101]:
from langchain.chains import LLMChain

vio_model = LLMChain(
    llm = llm,
    prompt = prompt_template ,
)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [102]:
def query_rag(query:str):
  similarity_search_results = db.similarity_search_with_score(query, k=2)
  result = []
  for doc,score in similarity_search_results:
    result.append(doc.page_content)

  context_text = "\n".join(result)
  rag_response = vio_model.invoke({"similarity_Context": context_text, "Violation": query})

  return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [103]:
user_query = "ماهي الغرامة على القيادة بدون رخصة؟"

In [104]:
response = query_rag(user_query)
response

{'similarity_Context': '<p><strong>عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.</strong> - الغرامة المالية 150 - 300 ريال</p>\n<p><strong>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</strong> - الغرامة المالية 1000 - 2000 ريال</p>',
 'Violation': 'ماهي الغرامة على القيادة بدون رخصة؟',
 'text': '150-300 ريال.'}